# Rotinas para Controle e manutenção de Grupos de WhatsApp 

In [1]:
import pandas as pd
import time

from datetime import datetime

# Ativar browser CHROMEDRIVER e autenticar QR Code do WhatsApp

In [2]:
#Personalisados (GrupoFramework: Funções de apoio a gerencia de grupos de Whatsapp via web)
from LoadBrowser import *
from NavBrowser  import *


In [ ]:
driver = InicioBrowser()
driver

# Data de registro da contagem de participantes
Será usado para criar a planilha com o resultado 
Geralmente faço uma contagem por dia e acumulo os dados para criação de dados e gráficos

In [ ]:
myDate = datetime.today()
myDate = myDate.strftime("%d/%m/%y %H:%M:%S")
myDate

# Importar lista de grupos 

Formato:
--------
Novo/ Antigo/ Link

Sendo:
Novo - Nome do grupo<br>
Antigo - Nome antigo que será trocado pelo NOVO caso dispare o comando de troca<br>
Link - Link de entrada para o grupo<br>



In [ ]:
dfGrupos = pd.read_csv('C:\\Users\\jandi\\Desktop\\Projeto Batalha Espiritual\\Lista de Grupos de WhatsApp - Página1.csv')

In [ ]:
start_time = time.time()

lCount = [] 
lLinks = [] # Array com links de cada grupo
nCount = 0 # Quantidade de participantes no grupo
nSleep = 0.5 # Espera entre cada acesso

#Controla função de ler links dos grupos 
#Caso seja necessário atualizar dados da campanha do Joinzapp
bLerLinks = False

lGrupos = dfGrupos['Novo']

for sGrupo in lGrupos:
    #Caixa de busca com argumento de nome do grupo para diminuir o div que é dinâmico, 
    #assim podemos acessar diretamente o grupo desejado
    search_box = AcharElemento(driver, '/html/body/div[1]/div/div/div[3]/div/div[1]/div/label/div/div[2]')
    if search_box == 'ElementoNaoEncontrado':
        print('Grupo não encontrado! [' + sGrupo + ']')
        continue
    search_box.clear()
    search_box.send_keys(sGrupo)
    time.sleep(nSleep)       

    #Seleciona o grupo da lista
    sGruponaLista = AcharElemento(driver, '//*[@title="' + sGrupo + '"]') 
    if sGruponaLista == 'ElementoNaoEncontrado':
        print('Grupo não encontrado! [' + sGrupo + ']')
        continue
    sGruponaLista.click()
    time.sleep(nSleep)

    #Clica no nome do grupo para abrir a aba de dados
    oNomedoGrupo = AcharElemento(driver, '/html/body/div[1]/div/div/div[4]/div/header/div[2]/div[1]/div/span') 
    if oNomedoGrupo == 'ElementoNaoEncontrado':
        print('Grupo não encontrado! [' + sGrupo + ']')
        continue
    oNomedoGrupo.click()
    time.sleep(nSleep)
    
    #Le a quantidade de pessoas no grupo
    sCount = AcharElemento(driver, '//*[@id="app"]/div/div/div[2]/div[3]/span/div/span/div/div/div[1]/div[5]/div[1]/div/div/div[1]/span')
    if sCount == 'ElementoNaoEncontrado':
        print('Quantidade de participantes não encontrada! [' + sGrupo + ']')
        continue
    
    sCount = sCount.text
    time.sleep(nSleep)

    if bLerLinks:
        #Le o link do grupo
        #Acrescentei o link aqui para controle do cadastro da ferramenta Joinzapp que faz administração de vários grupos
        sLink = AcharElemento(driver, '/html/body/div[1]/div/div/div[2]/div[3]/span/div/span/div/div/div[1]/div[5]/div[3]/div[2]')  
        if sLink == 'ElementoNaoEncontrado':
            print('Link do grupo não encontrado! [' + sGrupo + ']')
            continue
        sLink.click()

        sLink = AcharElemento(driver, '/html/body/div[1]/div/div/div[2]/div[3]/span/div/span/div/div/div[1]/div[2]/div[2]/div/span/a')
        if sLink == 'ElementoNaoEncontrado':
            print('Link do grupo não encontrado! [' + sGrupo + ']')
            continue

        sLink = sLink.text()    
        time.sleep(nSleep)
        lLinks.append(sLink)

    nCount = [int(algarismo) for algarismo in sCount.split() if algarismo.isdigit()][0]
    lCount.append(nCount)

print("--- %s minutes ---" % ( round( ((time.time() - start_time)/60), 2) ))

In [ ]:
dData = {
    'data': myDate,
    'grupos': lGrupos, 
    'participantes': lCount
}

In [ ]:
QtdeParticipantesPorGrupo = pd.DataFrame(dData)

if bLerLinks:
    QtdeParticipantesPorGrupo['Links'] = lLinks

# Verificar total de participantes em todos os grupos

In [ ]:
QtdeParticipantesPorGrupo['participantes'].sum()

# Exportar dataframe para planilha em Excel

In [ ]:
with pd.ExcelWriter('output.xlsx') as writer: 
     QtdeParticipantesPorGrupo.to_excel(writer, sheet_name='PartipantesGrupo')